## A test for applying BERT to cloze task

According to the answer from github [How can I apply Bert to a cloze task](https://github.com/huggingface/transformers/issues/80#issuecomment-444445782)

In [1]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text = 'From Monday to Friday most people are busy working or studying, '\
       'but in the evenings and weekends they are free and _ themselves.'
tokenized_text = tokenizer.tokenize(text)

masked_index = tokenized_text.index('_')
tokenized_text[masked_index] = '[MASK]'

candidates = ['love', 'work', 'enjoy', 'play']
candidates_ids = tokenizer.convert_tokens_to_ids(candidates)

indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

segments_ids = [0] * len(tokenized_text)

tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

language_model = BertForMaskedLM.from_pretrained('bert-base-uncased')
language_model.eval()

predictions = language_model(tokens_tensor, segments_tensors)
predictions_candidates = predictions[0, masked_index, candidates_ids]
answer_idx = torch.argmax(predictions_candidates).item()

print(f'The most likely word is "{candidates[answer_idx]}".')

The most likely word is "enjoy".


## Experiment of applying AUM to Bert based method

In [29]:
from pytorch_pretrained_bert import BertForSequenceClassification
from pytorch_pretrained_bert import BertConfig
import torch
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

In [45]:
def Trainer(batch_size, epoch = 50, n_classes = 2, vocab_size = 30522):
    writer = SummaryWriter('runs/NLP_AUM_First')
    
    config = BertConfig(vocab_size)
    model = BertForSequenceClassification(config, n_classes)
    dataset = get_dataset()
    dataloader = get_dataloader()
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = optim.Adam(lr = 0.0001)
    aum = torch.zeros([epoch, len(dataset), n_classes])
    
    for e in range(epoch):
        logits_store = torch.zeros([len(dataset), n_classes])
        for idx, batch in enumerate(dataloader):
            sample_idx, text_in, labels = batch
            logits = model(text_in)
            
            logits_store[sample_idx] = logits
            
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
        logits_topk, logits_topk_ind = torch.topk(logits_store, 2, 0)
        aum = logits_store - logits_topk[0][None]
        aum[epoch, logits_topk_ind[0,0], 0] = logits[logits_topk_ind[0,0], 0] - logits_topk[1, 0]
        aum[epoch, logits_topk_ind[0,1], 1] = logits[logits_topk_ind[0,1], 1] - logits_topk[1, 1]
        
        attention_index = #Need the attention index to point out the flipped sample
        
        for ind in attention_index:
            writer.add_scalar("Sentence {}".format(ind), aum[ind, ind_cls])